In [65]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [66]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [67]:
! nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


# MongoDB

In [68]:
from pymongo import MongoClient
import pymongo

In [69]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2014 = db.rm_patent_cn_2014
    db_rm_us_2014 = db.rm_patent_us_2014
    db_rm_de_2014 = db.rm_patent_de_2014

    
except errors.ConnectionFailure as err:
    print(err)

# 2014

## Company數量

In [70]:
data_assignee = db_rm_de_2014.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [71]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [72]:
data_assignee = db_rm_cn_2014.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [73]:
data_assignee = db_rm_us_2014.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [74]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [75]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
107,Cisco Technology Inc,85
239,Intel Corp,70
390,Qualcomm Inc,65
344,Nanjing IoT Sensor Technology Co Ltd,63
276,Jiangsu IoT Research and Development Center,54
522,Telefonaktiebolaget LM Ericsson AB,41
442,Samsung Electronics Co Ltd,25
114,Convida Wireless LLC,21
298,LG Electronics Inc,21
502,Suzhou Hickman IOT Technology Co. Ltd.,18


In [76]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [77]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Cisco Technology Inc,85,4.442651
1,Intel Corp,70,4.248495
2,Qualcomm Inc,65,4.174387
3,Nanjing IoT Sensor Technology Co Ltd,63,4.143135
4,Jiangsu IoT Research and Development Center,54,3.988984
5,Telefonaktiebolaget LM Ericsson AB,41,3.713572
6,Samsung Electronics Co Ltd,25,3.218876
7,Convida Wireless LLC,21,3.044522
8,LG Electronics Inc,21,3.044522
9,Suzhou Hickman IOT Technology Co. Ltd.,18,2.890372


公司名稱正規化

In [78]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Co. Ltd.","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("..","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba")\
                                                    .replace("ZTE Intelligent IoT Technology","ZTE"))

In [79]:
com_df

,company,count,count_log
0,Cisco Technology,85,4.442651
1,Intel,70,4.248495
2,Qualcomm,65,4.174387
3,Nanjing IoT Sensor Technology,63,4.143135
4,Jiangsu IoT Research and Development Center,54,3.988984
5,Telefonaktiebolaget LM Ericsson AB,41,3.713572
6,Samsung Electronics,25,3.218876
7,Convida Wireless LLC,21,3.044522
8,LG Electronics,21,3.044522
9,Suzhou Hickman IOT Technology,18,2.890372


In [80]:
com_ls = list(com_df.company)

# Data 2014

富比士2000資料

In [81]:
f_data = pd.read_csv("./Forbes/Forbes_2014.csv")

In [82]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Sector,Industry,Continent,Country,Market Value,Sales,Profits,Assets,Rank,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
21,Samsung Electronics,Information Technology,Semiconductors,Asia,South Korea,186.5,208.9,27.2,202.8,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Intel,Information Technology,Semiconductors,North America,United States,129.2,52.7,9.6,92.4,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Qualcomm,Information Technology,Semiconductors,North America,United States,135.5,25.5,6.8,46.3,154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,Taiwan Semiconductor,Information Technology,Semiconductors,Asia,Taiwan,102.0,20.1,6.3,42.4,190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,Texas Instruments,Information Technology,Semiconductors,North America,United States,51.4,12.2,2.1,18.9,421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,SK Hynix,Information Technology,Semiconductors,Asia,South Korea,24.9,12.9,2.6,19.8,439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
487,Micron Technology,Information Technology,Semiconductors,North America,United States,25.9,11.3,1.8,19.8,488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
589,ASML Holding,Information Technology,Semiconductors,Europe,Netherlands,41.2,7.0,1.6,16.8,591,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
866,Applied Materials,Information Technology,Semiconductors,North America,United States,25.0,8.1,0.5,12.2,868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,Broadcom,Information Technology,Semiconductors,North America,United States,18.3,8.2,0.4,11.5,964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [84]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [85]:
com_df[com_df.company.isin(Ind_ls)]

,company,count,count_log
1,Intel,70,4.248495
2,Qualcomm,65,4.174387
6,Samsung Electronics,25,3.218876


In [95]:
# pbc 0.1, 0.1
comp_ls = ["Suzhou Hickman IOT Technology","Zhuhai Youte Iot Technology","ZTE","Convida Wireless LLC","LG Electronics","Nokia Technologies Oy","Microsoft Technology Licensing LLC","Jiangsu IoT Research and Development Center","Belkin International"]

In [96]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [97]:
com_df[com_df.company.isin(comp_ls)]

,company,count,count_log
4,Jiangsu IoT Research and Development Center,54,3.988984
7,Convida Wireless LLC,21,3.044522
8,LG Electronics,21,3.044522
9,Suzhou Hickman IOT Technology,18,2.890372
10,Nokia Technologies Oy,15,2.708050
11,Zhuhai Youte Iot Technology,15,2.708050
12,ZTE,13,2.564949
13,Belkin International,12,2.484907
15,Microsoft Technology Licensing LLC,12,2.484907


In [98]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.08120446864154769
ari:  -0.08208955223880597


## 使用Forbes做join

In [99]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [100]:
# pbc 0.1, 0.1
comp_ls = ["Suzhou Hickman IOT Technology","Zhuhai Youte Iot Technology","ZTE","Convida Wireless LLC","LG Electronics","Nokia Technologies Oy","Microsoft Technology Licensing LLC","Jiangsu IoT Research and Development Center","Belkin International"]

In [101]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [102]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.000349860863659421
ari:  -0.0018495801194053298
